In [126]:
import requests
from bs4 import BeautifulSoup
from concurrent.futures import ThreadPoolExecutor
import time
import csv
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
driver = webdriver.Firefox(executable_path=r"geckodriver")


In [127]:
with open('products.csv', mode='w',  encoding='utf8') as products:
    fieldnames = ['Category', 'Subcategory', 'Title', 'Brand', 'Country', 'Url of Product']
    writer = csv.DictWriter(products,extrasaction='ignore', fieldnames=fieldnames)
    writer.writeheader()
    start = time.time()
    driver.get('https://contentservice.kz/produkty-pitaniya-napitki-kz/')
    page1 = driver.page_source
    lxml = BeautifulSoup(page1, "lxml")
    brands = {}
    for li in lxml.find('ul',class_ = 'link-list link-list_md').find_all('a'):
        driver.get('https://contentservice.kz/'+ str(li['href']))
        inner_pages = driver.page_source
        lxml2 = BeautifulSoup(inner_pages, "lxml")
        category = str(li.text)
        for li in lxml2.find('ul',class_ = 'link-list link-list_md').find_all('a'):
            driver.get('https://contentservice.kz/'+ str(li['href']))
            inner_inner_pages = driver.page_source
            lxml3 = BeautifulSoup(inner_inner_pages, "lxml")
            subcategory = str(li.text)
            while True:
                try:
                    load_next = driver.find_element_by_class_name('category-products-loadmore')
                    load_next.click()
                    time.sleep(0.2)
                except NoSuchElementException:
                    break
            #re-render page
            inner_inner_pages = driver.page_source
            lxml3 = BeautifulSoup(inner_inner_pages, "lxml")
            products = lxml3.find('div',class_='catalog__grid-view').find_all('a',class_='link-gray')
            for product in products:
                brand = product.find('span').text
                if (brand) not in brands:
                    product_desc = BeautifulSoup((requests.get(('https://contentservice.kz'+product['href'])).text),"lxml")
                    country = product_desc.find('span', class_='flag')
                    try:
                        brands[brand] = str(country['title'])
                    except:
                        brands[brand] = None
                    
                
                writer.writerow({'Category' : category,
                                'Subcategory' : subcategory,
                                'Title' : product.text, 
                                'Brand' : brand,
                                'Country' : brands.get(brand),
                                'Url of Product':('https://contentservice.kz' + product['href'])})
print('time scraped:', time.time()-start, 'seconds')

time scraped: 948.9403295516968 seconds


In [123]:
brands

{'Zeren': 'KZ',
 'Kokshe': 'KZ',
 'Nәtige': 'KZ',
 'Активиа': 'RU',
 'Danone': 'KZ',
 'Bio Max': 'RU',
 'Эмиль': 'KZ',
 'Чудо': 'RU',
 'Epica': 'RU',
 'Ehrmann': 'RU',
 'Food Master': 'KZ',
 'Растишка': 'KZ',
 'Actimel': 'RU',
 'Campina': 'RU',
 'Услада': 'RU',
 'Даниссимо': 'RU',
 'Био Баланс': 'KZ',
 'No Brand': 'KZ',
 'Агуша': 'RU',
 'Имунеле': 'RU',
 'Актуаль': 'RU',
 'Amiran': 'KZ',
 'Adal': 'KZ',
 'Умут и Ко': 'KG',
 'Lactis': 'KZ',
 'АЯ': 'KZ',
 'Текели': 'KZ',
 'Кефирные традиции': 'KZ',
 'Околица': 'KZ',
 'Свежее утро': 'KZ',
 'Брест-Литовск': 'BY',
 '': 'KZ',
 'Достойный выбор': 'KZ',
 'Маслёнково': 'KZ',
 'President': 'RU',
 'Петропавловское': 'KZ',
 'Златые горы': 'LT',
 'Домик в деревне': 'RU',
 'Масло-дел': 'KZ',
 'Молочный мир': 'KZ',
 'Крестьянское (масло)': 'KZ',
 'Эталон вкуса': 'KZ',
 'Простоквашино': 'RU',
 'Шин-Лайн': 'KZ',
 'Просто': 'KZ',
 'Наше': 'KZ',
 'Ne moloko': 'RU',
 'Кокшетауский молочный завод №1': 'KZ',
 'Молочная сказка': 'KZ',
 'Бурненское молоко': 'K

In [124]:
import pandas as pd
df = pd.read_csv('products.csv')
df

,Category,Subcategory,Title,Brand,Country,Url of Product
0,Молочные продукты и яйца 898,Йогурт 168,Йогурт 2.5% из козьего молока Zeren ст 200мл Z...,Zeren,KZ,https://contentservice.kz/product/710535169287...
1,Молочные продукты и яйца 898,Йогурт 168,Йогурт 4% из козьего молока Zeren п/бут 300мл ...,Zeren,KZ,https://contentservice.kz/product/710535169089...
2,Молочные продукты и яйца 898,Йогурт 168,Йогурт 4% из козьего молока Клубника Zeren п/б...,Zeren,KZ,https://contentservice.kz/product/710535169225...
3,Молочные продукты и яйца 898,Йогурт 168,Йогурт 3% Турецкий Kokshe ст 360г Kokshe,Kokshe,KZ,https://contentservice.kz/product/487023543001...
4,Молочные продукты и яйца 898,Йогурт 168,Йогурт 1.5% Клубника-малина Nәtige т/п 450г Nә...,Nәtige,KZ,https://contentservice.kz/product/487000375130...
...,...,...,...,...,...,...
5861,Табачные изделия 72,Сигареты 71,Сигареты с фильтром State Line 20шт State Line,State Line,KZ,https://contentservice.kz/product/48743914-kz-...
5862,Табачные изделия 72,Сигареты 71,Сигареты с фильтром Winston Silver 20шт Winston,Winston,NaN,https://contentservice.kz/product/48741200-kz-...
5863,Табачные изделия 72,Сигареты 71,СИГАРЕТЫ PHILIP MORRIS PARLIAMENT RESERVE КОР ...,Parliament,KZ,https://contentservice.kz/product/48741514-kz-...
5864,Табачные изделия 72,Сигареты 71,Сигареты с угольным фильтром Kent HDi Silver 2...,Kent,RU,https://contentservice.kz/product/46110312-kz-...
